In [1]:
import pandas as pd
import numpy as np
import pickle
import os, time
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [3]:
# Reading CSV files, and merging all of them into a single DataFrame
CISIDS2017_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CICIDS2017ML"
CTU13_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CTU13ML"

CICIDS2017_df_list = []
CTU13_df_list = []

# Reading CICIDS2017 CSV files into a single DataFrame
for f in os.listdir(CISIDS2017_folder):
    file_path = os.path.join(CISIDS2017_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CICIDS2017_df_list.append(pd.read_csv(file_path))

# Reading CTU13 CSV files into a single DataFrame
for f in os.listdir(CTU13_folder):
    file_path = os.path.join(CTU13_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CTU13_df_list.append(pd.read_csv(file_path))

CICIDS2017_df, CTU13_df = pd.concat(CICIDS2017_df_list, ignore_index=True), pd.concat(CTU13_df_list, ignore_index=True)

Reading:  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Morning.pcap_ISCX_Relabeled.csv
Reading:  Wednesday-workingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_Relabeled.csv
Reading:  Monday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_Relabeled.csv
Reading:  Tuesday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_Relabeled.csv
Reading:  CTU13_Normal_Traffic_Relabeled.csv
Reading:  CTU13_Attack_Traffic_Relabeled.csv


In [4]:
# QUICK PREPROCESSING. 
# Some classifiers do not like "infinite" (inf) or "null" (NaN) values.
CICIDS2017_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CICIDS2017_df.columns[CICIDS2017_df.isna().any()]))
CICIDS2017_df.dropna(inplace=True)

CTU13_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CTU13_df.columns[CTU13_df.isna().any()]))
CTU13_df.dropna(inplace=True)

Columns with problematic values:  [' Flow Packets/s']
Columns with problematic values:  []


In [5]:
CICIDS2017_df[' Label'].unique()

array(['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'FTP-Patator', 'SSH-Patator',
       'Infiltration'], dtype=object)

In [6]:
CTU13_df[' Label'].unique()

array(['BENIGN', 'Bot'], dtype=object)

In [7]:
# Create a new column that unifies all malicious classes into a single class for binary classification
CICIDS2017_df['GT'] = np.where(CICIDS2017_df[' Label']=='BENIGN', 'Benign', 'Malicious')
CTU13_df['GT'] = np.where(CTU13_df[' Label']=='BENIGN', 'Benign', 'Malicious')

In [8]:
features = pd.Index([
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max', 
    ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean',
    ' Fwd Packet Length Std',
    ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean',
    ' Bwd Packet Length Std',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Flow IAT Std',
    ' Flow IAT Max',  
    ' Flow IAT Min',
    ' Fwd IAT Mean',
    ' Fwd IAT Std',
    ' Fwd IAT Max',
    ' Fwd IAT Min',
    ' Bwd IAT Mean',
    ' Bwd IAT Std',
    ' Bwd IAT Max',
    ' Bwd IAT Min',
    ' Bwd PSH Flags',
    ' Fwd Header Length',
    ' Bwd Header Length',
    ' Bwd Packets/s',
    ' Min Packet Length',
    ' Max Packet Length',
    ' Packet Length Mean',
    ' Packet Length Std', 
    ' Packet Length Variance',
    ' SYN Flag Count',
    ' RST Flag Count',
    ' ACK Flag Count',
    ' Down/Up Ratio',
    ' Average Packet Size', 
    ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size',
    ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd',
    ' Active Std',
    ' Active Max',
    ' Active Min', 
    ' Idle Std',
    ' Idle Max',
    ' Idle Min' 
])

In [22]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Train a binary Dummy Classifier model on CTU13 dataset
start = time.time()
dummy_bin_ctu13 = DummyClassifier(strategy='stratified')
dummy_bin_ctu13.fit(train_ctu13[features], train_ctu13['GT'])
end = time.time() - start
print("Training time (CTU13 Binary): ", end)

# Save the binary Dummy Classifier model trained on CTU13 dataset
with open('../Pickle Files/DummyBinaryCTU13.pkl', 'wb') as file:
    pickle.dump(dummy_bin_ctu13, file)

Training time (CTU13 Binary):  0.07322359085083008


In [23]:
# Train test splits for both datasets 
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Train a binary Dummy Classifier model on CICIDS2017 dataset
start = time.time()
dummy_bin_cicids = DummyClassifier(strategy='stratified')
dummy_bin_cicids.fit(train_cicids[features], train_cicids['GT'])
end = time.time() - start
print("Training time (CICIDS2017 Binary): ", end)

# Save the binary Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyBinaryCICIDS2017.pkl', 'wb') as file:
    pickle.dump(dummy_bin_cicids, file)

Training time (CICIDS2017 Binary):  4.4926745891571045


In [24]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Train a multiclass Dummy Classifier model on CICIDS2017 dataset
start = time.time()
dummy_multi_cicids = DummyClassifier(strategy='stratified')
dummy_multi_cicids.fit(train_cicids[features], train_cicids[' Label'])
end = time.time() - start
print("Training time (CICIDS2017 Multiclass): ", end)

# Save the multiclass Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyMulticlassCICIDS2017.pkl', 'wb') as file:
    pickle.dump(dummy_multi_cicids, file)

Training time (CICIDS2017 Multiclass):  1.6379647254943848


In [10]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT']) 
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CTU13 dataset
with open('../Pickle Files/DummyBinaryCTU13.pkl', 'rb') as file:
    dummy_bin_ctu13 = pickle.load(file)
    
# Test on CTU13 dataset
predictions_bin_ctu13 = dummy_bin_ctu13.predict(test_ctu13[features])
print("Acc (CTU13 Binary Dummy): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin_ctu13)))
print("Precision (CTU13 Binary Dummy): {:3f}".format(precision_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
print("Recall (CTU13 Binary Dummy): {:3f}".format(recall_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
print("F1-score (CTU13 Binary Dummy): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin_ctu13, rownames=['True'], colnames=['Pred'])

Acc (CTU13 Binary Dummy): 0.509069
Precision (CTU13 Binary Dummy): 0.417779
Recall (CTU13 Binary Dummy): 0.416222
F1-score (CTU13 Binary Dummy): 0.416999


Pred,Benign,Malicious
True,,
Benign,12301,9025
Malicious,9083,6476


In [11]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyBinaryCICIDS2017.pkl', 'rb') as file:
    dummy_bin_cicids = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = dummy_bin_cicids.predict(test_cicids[features]) 
print("Acc (CICIDS2017 Binary Dummy): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("Precision (CICIDS2017 Binary Dummy): {:3f}".format(precision_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
print("Recall (CICIDS2017 Binary Dummy): {:3f}".format(recall_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
print("F1-score (CICIDS2017 Binary Dummy): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary Dummy): 0.684271
Precision (CICIDS2017 Binary Dummy): 0.197278
Recall (CICIDS2017 Binary Dummy): 0.196880
F1-score (CICIDS2017 Binary Dummy): 0.197079


Pred,Benign,Malicious
True,,
Benign,730184,178344
Malicious,178793,43830


In [13]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the multiclass Dummy Classifier model trained on CICIDS2017 dataset
with open('../Pickle Files/DummyMulticlassCICIDS2017.pkl', 'rb') as file:
    dummy_multi_cicids = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_multi_cicids = dummy_multi_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Multiclass Dummy): {:3f}".format(accuracy_score(test_cicids[' Label'], predictions_multi_cicids)))
print("Precision (CICIDS2017 Multiclass Dummy): {:3f}".format(precision_score(test_cicids[' Label'], predictions_multi_cicids, average='weighted')))
print("Recall (CICIDS2017 Multiclass Dummy): {:3f}".format(recall_score(test_cicids[' Label'], predictions_multi_cicids, average='weighted')))
print("F1-score (CICIDS2017 Multiclass Dummy): {:3f}".format(f1_score(test_cicids[' Label'], predictions_multi_cicids, average='weighted')))
pd.crosstab(test_cicids[' Label'], predictions_multi_cicids, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Multiclass Dummy): 0.656698
Precision (CICIDS2017 Multiclass Dummy): 0.656917
Recall (CICIDS2017 Multiclass Dummy): 0.656698
F1-score (CICIDS2017 Multiclass Dummy): 0.656808


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,,,
BENIGN,729376,602,41362,3414,73940,1751,1811,2585,2,15,50992,1948,514,5,211
Bot,626,0,34,4,63,1,3,1,0,0,49,1,0,0,0
DDoS,40959,40,2335,173,4213,92,105,152,0,1,2835,109,28,0,17
DoS GoldenEye,3368,3,189,15,322,5,10,10,0,0,220,9,3,0,0
DoS Hulk,73839,69,4177,367,7527,165,219,244,0,1,5097,176,58,0,32
DoS Slowhttptest,1731,2,115,2,190,6,7,3,0,0,141,5,0,0,1
DoS slowloris,1818,0,98,10,165,5,1,6,0,0,138,4,0,0,0
FTP-Patator,2532,1,133,22,239,10,11,13,0,0,200,7,4,0,3
Heartbleed,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [14]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CTU13 dataset
with open('../Pickle Files/DummyBinaryCTU13.pkl', 'rb') as file:
    dummy_bin_ctu13 = pickle.load(file)

# Test on CICIDS2017 dataset
predictions_bin_cicids = dummy_bin_ctu13.predict(test_cicids[features])
print("Acc (CTU13 to CICIDS2017 Dummy): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("Precision (CTU13 to CICIDS2017 Dummy): {:3f}".format(precision_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
print("Recall (CTU13 to CICIDS2017 Dummy): {:3f}".format(recall_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
print("F1-score (CTU13 to CICIDS2017 Dummy): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

Acc (CTU13 to CICIDS2017 Dummy): 0.547459
Precision (CTU13 to CICIDS2017 Dummy): 0.196968
Recall (CTU13 to CICIDS2017 Dummy): 0.422288
F1-score (CTU13 to CICIDS2017 Dummy): 0.268636


Pred,Benign,Malicious
True,,
Benign,525248,383280
Malicious,128612,94011


In [15]:
# Train test splits for both datasets
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_cicids, test_cicids = train_test_split(CICIDS2017_df, test_size=0.4, stratify=CICIDS2017_df['GT'])

# Load the binary Dummy Classifier model trained on CICIDS2017 dataset 
with open('../Pickle Files/DummyBinaryCICIDS2017.pkl', 'rb') as file:
    dummy_bin_cicids = pickle.load(file)

# Test on CTU13 dataset
predictions_bin = dummy_bin_cicids.predict(test_ctu13[features])
print("Acc (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin)))
print("Precision (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(precision_score(test_ctu13['GT'], predictions_bin, pos_label='Malicious')))
print("Recall (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(recall_score(test_ctu13['GT'], predictions_bin, pos_label='Malicious')))
print("F1-score (CICIDS2017 Binary to CTU13 Dummy): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin, rownames=['True'], colnames=['Pred'])

Acc (CICIDS2017 Binary to CTU13 Dummy): 0.545886
Precision (CICIDS2017 Binary to CTU13 Dummy): 0.418614
Recall (CICIDS2017 Binary to CTU13 Dummy): 0.196864
F1-score (CICIDS2017 Binary to CTU13 Dummy): 0.267792


Pred,Benign,Malicious
True,,
Benign,17072,4254
Malicious,12496,3063
